In [24]:
import requests

standings_2024_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [16]:
#Get HTML from server
data = requests.get(standings_2024_url)

In [ ]:
#Grab all href tags from <a> in the standings table, as we want data from all teams
from bs4 import BeautifulSoup

#Initialize soup object
soup = BeautifulSoup(data.text)
soup

In [18]:
#Select the table element using table.stats_table css selector, specifically select the first table
standings_table = soup.select('table.stats_table')[0]


IndexError: list index out of range

In [5]:
links = standings_table.find_all('a')
links = [l.get("href") for l in links]

#Want to remove all links to do with players and only have links to do with clubs
links = [l for l in links if "/squads" in l] #Include only links with "/squad" inside the link
links #Note that all of these urls are relative paths

NameError: name 'standings_table' is not defined

In [6]:
#Add full paths by concatenating string to all
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls


NameError: name 'links' is not defined

In [50]:
arsenal_url = team_urls[5]
#Get HTML from Arsenal page
arsenal_data = requests.get(arsenal_url)

# arsenal_fixtures = soup.select('table.stats_table')[0]




In [25]:
import pandas as pd

#Use HTML to read into pd df, pd will detect tables
#Match parameter - The set of tables containing text matching this regex or string will be returned.
matches = pd.read_html(arsenal_data.text, match="Scores & Fixtures")[0]

NameError: name 'arsenal_data' is not defined

# Extract shooting stats from each match

In [52]:
soup = BeautifulSoup(arsenal_data.text)
links = soup.find_all("a") #Get all the <a> tags, links at this point includes array full of a tags, we want to get the urls from these a tags (href from a)
links = [l.get("href") for l in links ] #Get all the hrefs
links = [l for l in links if l and 'all_comps/shooting' in l]
links


[]

In [53]:
shooting_html = requests.get(f"https://fbref.com{links[0]}")
shooting_table = pd.read_html(shooting_html.text, match="Shooting")[0]
shooting_table.head() #From head here we can see that there is multi-index level as there are two rows of columns, generally this is not necessary

#We want to be able to index by the column names and so we can drop level
shooting_table.columns = shooting_table.columns.droplevel()
shooting_table.columns

IndexError: list index out of range

In [54]:
shooting_cols = ["Date", "Sh", "SoT", "SoT%", "G/Sh", "G/SoT", "FK", "PK", "xG"]

match_shape = matches.shape
shooting_shape = shooting_table.shape
print(match_shape, shooting_shape) #(52,19) and (53,26), our dataset is not perfect and for some reason there is a match that is not logged in the table


#With the above in mind when we merge we will only combine the matching dates in both
team_data = matches.merge(shooting_table[shooting_cols], on="Date")
team_data.head()
# print(team_data.shape) #Shape is (52,27)

(51, 19) (52, 26)


,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,SoT%,G/Sh,G/SoT,FK,PK,xG_y
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Home,D,1,1,Liverpool,...,Match Report,NaN,10,4,40.0,0.10,0.25,1.0,0,1.4
1,2023-08-20,16:30,Premier League,Matchweek 2,Sun,Away,L,1,3,West Ham,...,Match Report,NaN,16,3,18.8,0.06,0.33,0.0,0,2.5
2,2023-08-25,20:00,Premier League,Matchweek 3,Fri,Home,W,3,0,Luton Town,...,Match Report,NaN,19,8,42.1,0.16,0.38,1.0,0,2.2
3,2023-08-30,19:45,EFL Cup,Second round,Wed,Home,W,2,1,AFC Wimbledon,...,Match Report,NaN,23,9,39.1,0.04,0.11,NaN,1,NaN
4,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,L,0,1,Nott'ham Forest,...,Match Report,NaN,21,2,9.5,0.00,0.00,2.0,0,2.3


## Scrape data for multiple teams across multiple seasons

In [26]:
years = list(range(2022,2020, -1))
years

[2022, 2021]

In [27]:
all_matches = []

In [28]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [30]:
import time
import requests
import pandas as pd

shooting_cols = ["Date", "Sh", "SoT", "SoT%", "G/Sh", "G/SoT", "FK", "PK", "xG"]

for year in years: #Scrape each season
  data = requests.get(standings_url)
  soup = BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[0]

  links = standings_table.find_all('a')
  links = [l.get("href") for l in links]
  links = [l for l in links if "/squads" in l]

  team_urls = [f"https://fbref.com{l}" for l in links]

  for team_url in team_urls: #Scrape each team in a given season
    team_name = team_url.split("/")[-1].replace("-Stats","").replace("-","") #Get the team name
    team_html = requests.get(team_url)
    soup = BeautifulSoup(team_html.text)
    print(team_name)

    matches = pd.read_html(team_html.text, match="Scores & Fixtures")[0] #Get table of match data

    links = soup.find_all('a')
    links = [l.get("href") for l in links ]
    links = [l for l in links if l and 'all_comps/shooting' in l] #Get shooting links

    shooting_html = requests.get(f"https://fbref.com{links[0]}") #Convert to absolute url
    shooting_table = pd.read_html(shooting_html.text, match="Shooting")[0]
    shooting_table.head()
    shooting_table.columns = shooting_table.columns.droplevel()

    try:
      team_data = matches.merge(shooting_table[shooting_cols], on="Date") #Some shooting stats not available for all teams
    except ValueError:
      continue

    team_data[team_data["Comp"] == "Premier League"] #We are predicting premier league so for now we only want data in Premier League

    #Make sure you preserve important data
    team_data["Season"] = year
    team_data["Name"] = team_name

    all_matches.append(team_data)
    time.sleep(1) #Good practice so that you dont scrape too quickly and slow down website


KeyboardInterrupt: 

match_df = pd.concat(all_matches)
all_matches

In [59]:
match_df.to_csv("matches.csv")

NameError: name 'match_df' is not defined